In [ ]:
import boto3
import os
import json
import numpy as np
import cv2
from src.utils import resize_image

from src.sm_utils import get_latest_training_job, get_model_data, parse_model_data

# Build an endpoint with the new models
In this notebook, we build new SageMaker endpoints with the models that you trained.

In [ ]:
with open("stack_outputs.json") as f:
    sagemaker_config = json.load(f)
s3_bucket = sagemaker_config["S3Bucket"]

## Get the models from S3

In [ ]:
os.mkdir("model")

word_seg_training_job = get_latest_training_job(sagemaker_config["SolutionPrefix"]+"-word-seg")
word_seg_s3 = get_model_data(word_seg_training_job)
parse_model_data(word_seg_s3, "model")

htr_training_job = get_latest_training_job(sagemaker_config["SolutionPrefix"]+"-line-reg")
htr_s3 = get_model_data(htr_training_job)
parse_model_data(htr_s3, "model")

!tar -czvf model.tar.gz model/
!aws s3 cp model.tar.gz s3://$s3_bucket/custom_data/artifacts/model.tar.gz

In [ ]:
# Clean up the temporary folders and files
!rm -r model
!rm output.tar.gz
!rm model.tar.gz

## Build new endpoints with the models

In [ ]:
import sagemaker

from sagemaker.mxnet.model import MXNetModel

role = sagemaker_config["SageMakerIamRole"]

mxnet_model = MXNetModel(model_data='s3://{}/custom_data/artifacts/model.tar.gz'.format(s3_bucket),
                         role=role,
                         entry_point='inference.py',
                         source_dir="src",
                         framework_version='1.6.0',
                         name=sagemaker_config["SageMakerModelName"],
                         code_location="s3://{}/endpoint".format(s3_bucket),
                         py_version='py3')

predictor = mxnet_model.deploy(instance_type=sagemaker_config["SageMakerInferenceInstanceType"],
                               endpoint_name=sagemaker_config["SageMakerEndpointName"],
                               initial_instance_count=1)

In [ ]:
img = cv2.imread('data/sample/demo.png', cv2.IMREAD_GRAYSCALE)

resized_img, _ = resize_image(img, desired_size=(700, 700))
out = predictor.predict(resized_img.astype('float32').tolist())

line_bbs = np.array(out["line_bbs"])
decoded = out["decoded"]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

fig, ax = plt.subplots(1, figsize=(20, 15))

for j in range(line_bbs.shape[0]):
    decoded_string = decoded[j]
    (x, y, w, h) = line_bbs[j]
    image_h, image_w = resized_img.shape[-2:]
    (x, y, w, h) = (x * image_w, y * image_h, w * image_w, h * image_h)

    ax.text(x+w, y+h/2, decoded_string, bbox=dict(facecolor='red', alpha=0.5))
    rect = patches.Rectangle((x, y), w, h, fill=False, color="r")

    ax.add_patch(rect)
    
ax.set_title("Predicted image")
ax.imshow(resized_img, cmap='Greys_r')
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)

# Navigation
- Click [here](./4_model_training.ipynb) to go back and retrain a model again.